In [1]:
!pip install nltk

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score as score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import make_pipeline
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
#from nltk.stem.snowball import EnglishStemmer
#from spacy.tokens import Doc
import en_core_web_sm
import re


data_train = pd.read_csv('E:\\PycharmProjects\\Machine_L\\Data_Science_Club\\second_\\train.csv')
data_test = pd.read_csv('E:\\PycharmProjects\\Machine_L\\Data_Science_Club\\second_\\test.csv')
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

all_text = pd.concat([data_train['comment_text'], data_test['comment_text']])

en_lp = en_core_web_sm.load()

def tokenizer_(doc):
    doc_spacy = en_lp(doc)
    return [token.lemma_ for token in doc_spacy]


class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

    

vectorizer_count = TfidfVectorizer(min_df = 0.2, max_df = 0.8)
vectorizer_count.fit(all_text)



TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=None, min_df=0.2,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [4]:
train_text_features_count = vectorizer_count.transform(data_train['comment_text'])
test_text_features_count = vectorizer_count.transform(data_test['comment_text'])

In [5]:
#pipe = make_pipeline(LogisticRegression(), CountVectorizer())
grid = {'alpha': np.power(10.0, np.arange(-5, 5))}
scores_vectorizer_count = []
best_params_vectorizer_count = []
for cl_name in class_names:
    y_train = data_train[cl_name]
    gs = GridSearchCV(Ridge(random_state = 241), grid, n_jobs = -1, scoring = 'roc_auc', cv=KFold(n_splits = 5, shuffle = True, random_state = 241))
    gs.fit(train_text_features_count, y_train)
    scores_vectorizer_count.append(gs.best_score_)
    best_params_vectorizer_count.append(gs.best_params_)
    for scores in gs.grid_scores_:
        print(scores.mean_validation_score)
        print(scores.parameters)

c:\python36\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


0.7723626589429857
{'alpha': 1e-05}
0.772362962811164
{'alpha': 0.0001}
0.7723628673840375
{'alpha': 0.001}
0.7723687486679814
{'alpha': 0.01}
0.7723628985094642
{'alpha': 0.1}
0.7723640596427802
{'alpha': 1.0}
0.7723623158844174
{'alpha': 10.0}
0.7723498408613904
{'alpha': 100.0}
0.7712858044387685
{'alpha': 1000.0}
0.7637257589287407
{'alpha': 10000.0}


KeyboardInterrupt: 

In [3]:
f = open('Cross-validation scores for CountVect(min_df = 5).txt', 'w')
f.write('LogRegression with default penalty and C-parameters founded by GridSearchCV')
f.write('\n')
for i in range(len(scores_vectorizer_count)):
    f.write(class_names[i])
    f.write(' ')
    f.write(str(best_params_vectorizer_count[i]))
    f.write(' ')
    f.write(str(scores_vectorizer_count[i]))
    f.write('\n')
f.write('\n')    
f.write('Mean of scores for all classes ')
f.write(str(np.mean(scores_vectorizer_count)))
f.close()

In [3]:
y_train_tox = data_train['identity_hate']
scor = np.mean(score(LogisticRegression(C = 1000, penalty = 'l2'), X = train_text_features_count, y = y_train_tox, scoring = 'roc_auc', cv=KFold(n_splits = 5, shuffle = True, random_state = 241)))

In [4]:
scor

0.898280529536678